# Solving Navier-Stokes with a stationary body

In [1]:
using ViscousFlow

In [ ]:
using Plots
pyplot()
default(grid = false)

Set the flow parameters

In [ ]:
Re = 200; # Reynolds number
U = 1.0; # Free stream velocity
U∞ = (U,0.0);

Set up the domain, grid spacing, and time step size

In [ ]:
xlim = (-1.0,3.0)
ylim = (-1.0,1.0)

Δx, Δt = setstepsizes(Re)

#### Set up points on the body. We set the points so that the spacing between them is 1.5 times the grid spacing.

Here is a plate:

In [ ]:
body = Plate(1.0,1.5Δx)

or here is a NACA 0012 airfoil

In [ ]:
#body = NACA4(0.0,0.0,0.12,1.5Δx)

#### Transform the body to a specified initial position and orientation.

In [ ]:
cent = (0.0,0.0)
α = 40π/180 #π/3 #π/4
T = RigidTransform(cent,-α)
T(body) # transform the body to the current configuration

In [ ]:
plot(body,xlim=xlim,ylim=ylim)

Collect the body point data into a vector

In [ ]:
X = VectorData(collect(body));

### Now set up the system

Set up the system operators

In [ ]:
sys = NavierStokes(Re,Δx,xlim,ylim,Δt,U∞ = U∞, X̃ = X, isstore = true)

Initial conditions, and grid coordinates

In [ ]:
w₀ = Nodes(Dual,size(sys));
f = VectorData(X);
xg, yg = coordinates(w₀,dx=cellsize(sys),I0=origin(sys))

Set up the integrator here

In [ ]:
plan_intfact(t,u) = CartesianGrids.plan_intfact(t,u,sys)
plan_constraints(u,t) = ConstrainedSystems.plan_constraints(u,t,sys)
r₁(u,t) = ConstrainedSystems.r₁(u,t,sys)
r₂(u,t) = ConstrainedSystems.r₂(u,t,sys)


@time solver = IFHERK(w₀,f,timestep(sys),plan_intfact,plan_constraints,(r₁,r₂),rk=ConstrainedSystems.RK31)

In [ ]:
w₀

Initialize the state vector and the history vectors

In [ ]:
t = 0.0
u = zero(w₀)

# Rate at which to store field data
tsample = 0.2

# Field data storage plan
S = StorePlan(tsample,"state"=>u,"time"=>t)
data_history = initialize_storage(S)

# Force data storage plan, stored every time step
F = StorePlan(timestep(sys),"fx"=>sum(f.u),"fy"=>sum(f.v),"time"=>t)
force_history = initialize_storage(F)
nothing

### Advance the system!

Set the length of time to integrate over.

In [ ]:
tsim = 1.0;

In [ ]:
for ti in timerange(tsim,sys)
    global t, u, f = solver(t,u)
    
    fds = force(f,sys) 
    
    store_data!(data_history,t,S,"state" => u, "time" => t)
    store_data!(force_history,t,F,"fx" => sum(fds.u),"fy" => sum(fds.v),"time" => t)
    
 end
println("solution completed through time t = ",t)

#### Plotting

Basic plot

In [ ]:
plot(xg,yg,vorticity(data_history["state"][end],sys),levels=range(-15,15,length=30), color = :RdBu,clim=(-15,15))
plot!(body)

Streamlines

In [ ]:
plot(xg,yg,streamfunction(data_history["state"][end],sys),levels=range(-1,1,length=31), color = :black,clim=(-0.25,0.25),xlim=(-1,3),ylim=(-1,1))
plot!(body)

Plot lift and drag coefficient

In [ ]:
plt = plot(layout = (2,1), size = (600, 400))
plot!(plt[1],force_history["time"],2*force_history["fy"],xlim=(0,6),ylim=(0,4),xlabel="Convective time",ylabel="\$C_L\$",legend=false)
plot!(plt[2],force_history["time"],2*force_history["fx"],xlim=(0,6),ylim=(0,4),xlabel="Convective time",ylabel="\$C_D\$",legend=false)
plt